In [ ]:
# Import packages
import os
from matplotlib import pyplot as plt
import pandas as pd
import datetime
import numpy as np
import math

# Import AuTuMN modules
from autumn.settings import Models
from autumn.settings.region import Region
from autumn.settings.folders import OUTPUT_DATA_PATH
from autumn.core.project import get_project
from autumn.core import db

from autumn.core.plots.plotter.base_plotter import COLOR_THEME
from autumn.core.plots.calibration.plots import get_posterior, get_epi_params, plot_posterior
from autumn.calibration.utils import get_uncertainty_df

In [ ]:
# Specify model details
model = Models.COVID_19
region = Region.SRI_LANKA


In [ ]:
def get_uncertainty_data(dirname):
    # get the relevant project and output data
    project = get_project(model, region)
    project_calib_dir = os.path.join(
        OUTPUT_DATA_PATH, "calibrate", project.model_name, project.region_name
    )
    calib_path = os.path.join(project_calib_dir, dirname)
    # Load tables
    mcmc_tables = db.load.load_mcmc_tables(calib_path)
    mcmc_params = db.load.load_mcmc_params_tables(calib_path)
    
    return mcmc_params, mcmc_tables

In [ ]:
def new_get_posterior(mcmc_params, mcmc_tables, param_name, burn_in=0):
    weighted_vals = []
    for param_df, run_df in zip(mcmc_params, mcmc_tables):
        table_df = param_df.merge(run_df, left_index=True,  right_index=True)
        unweighted_vals = table_df[param_name]
        weights = table_df.weight
        for v, w in zip(unweighted_vals, weights):
            weighted_vals += [v] * w

    return pd.DataFrame(weighted_vals, columns=[param_name])


In [ ]:
dirCHN = "2022-06-06" # China matrix
dirHKG = "2022-06-28" # Hong Kong matrix

param_name = "contact_rate"
burn_in = 500

mcmc_params_CHN, mcmc_tables_CHN = get_uncertainty_data(dirCHN)
mcmc_params_HKG, mcmc_tables_HKG = get_uncertainty_data(dirHKG)

vals_df_CHN = new_get_posterior(mcmc_params_CHN, mcmc_tables_CHN, param_name, burn_in)[param_name]
vals_df_HKG = new_get_posterior(mcmc_params_HKG, mcmc_tables_HKG, param_name, burn_in)[param_name]

In [ ]:
#plot the posterior distributions
num_bins = 16
fontsize =24
label_font_size =20

fig = plt.figure(figsize=(10,8))
axis = fig.add_subplot(1, 1, 1)

contact_rate_ratio = vals_df_CHN/vals_df_HKG
contact_rate_ratio.hist(bins=num_bins, ax=axis, density=True)

axis.set_xlabel("ratio of contact rate", fontsize = fontsize)
axis.set_ylabel("frequency", fontsize = fontsize)
plt.setp(axis.get_yticklabels(), fontsize=label_font_size)
plt.setp(axis.get_xticklabels(), fontsize=label_font_size)

newlist = [x for x in contact_rate_ratio if math.isnan(x) == False] #removing Nan

print(np.quantile(newlist,[0.5,0.025,0.975]))